# Astro 528, Lab 2, Exercise 2

## Numerical stability of N-body integrators

According to our reading, one of the top priorities for scientific computing is numerical stability.  It can be hard to appreciate why numerical stability is important.  In order to illustrate the importance of numerical stability, we will consider a classic problem, integrating the trajectory of a star and planet interacting under Newtonian gravity.  

This can be expressed as integrating a set of $3N$ second-order ordinary differential equations (ODEs).
$$ 
\frac{d^2 x_i}{dt^2} = \sum_{j=1}^N \frac{G m_j (x_j-x_i)}{\left|x_j-x_i\right|^3},
$$
where $x_i$ is the position vector of body $i$, $m_i$ is the mass of body $i$, $G$ is the gravitational constant, and $N$ is the number of bodies in the system.  
Typically, this is rewritten as a set of $6N$ first-order ODEs.
\begin{eqnarray}
\frac{dx_i}{dt} & = & v_i \\
\frac{dv_i}{dt} & = & \sum_{j=1}^N \frac{G m_j (x_j-x_i)}{\left|x_j-x_i\right|^3}
\end{eqnarray}
where the $v_i$'s are the velocity vectors.  

In this exercise, you will compare the results of integrating these equations using multiple different algorithms and parameters.  We'll start with simple integration algorithms that one could easily code on their own, but then progress to more sophisticated integration algorithms that would require a fair bit of work to implement oneself.  In order to allow you to focus on the main point of this exercise, we'll use Julia's [DifferentialEquations.jl](https://github.com/JuliaDiffEq/DifferentialEquations.jl).  The package has [good documentation](http://docs.juliadiffeq.org/stable/), but I'll provide nearly all the code you need to do the calculations.  Your job is to read through the code enough that you can tell what it's doing, to think about what you expect to happen, and then compare your expectations to the actual results.  If you read over the code, it may help you start to get comfortable with julia programming, but you don't need to worry about every little bit of syntax for now.  

First, we need to write a function that computes the derivatives of each of the positions and velocities.

In [ ]:
"""
   gravity_as_first_order_system(du, u, mass, t)

Computes velocities and accelerations due to Newtonian gravity
for a system of N massive particles.

# Inputs:  
- u: array of 3*N positions followed by 3*N velocities
  [x_1, y_1, z_1, x_2, y_2, z_2, ...., vx_1, vy_1, vz_1, ..., vz_N]
- mass: array of N masses
- t: time

# Outputs:
- du: Aray of 3*N velocities followed by 3*N accelerations
  [vx_1, vy_1, vz_1, vx_2, vy_2, vz_2, ...., ax_1, ay_1, az_1, ..., az_N]

# Assumptions:
- Gravitational constant = 1.  
Therefore, if masses are in solar masses and distances are in AU, 
then the time unit is such that one year = 2pi.

"""
function gravity_as_first_order_system(du, u, mass, t)
    @assert length(du) == length(u) == 6 * length(mass)
    N = length(mass)
    @assert 2 <= N      # require at least two massive bodies
    for k in 1:(3*N) 
       du[k] = u[3N+k]  # derivative of position is simply velocity
    end
    du[3N+1:end] .= 0.0   # initialize accelerations to zero
    for i in 0:(N-2)      # loop over each pairs of bodies
        for j in (i+1):(N-1)
            dx = u[3*i+1] - u[3*j+1]  # displacements
            dy = u[3*i+2] - u[3*j+2]
            dz = u[3*i+3] - u[3*j+3]
            # calculate distance once per pair
            d = sqrt(dx^2+dy^2+dz^2)  
            # derivatives of velocities are accelerations 
            # acceleration on body i due to body j
            du[3N+3*i+1] -= mass[j+1] * dx / d^3 
            du[3N+3*i+2] -= mass[j+1] * dy / d^3 
            du[3N+3*i+3] -= mass[j+1] * dz / d^3 
            # reuse computation of d for acceleration on body j due to body i
            du[3N+3*j+1] += mass[i+1] * dx / d^3 
            du[3N+3*j+2] += mass[i+1] * dy / d^3 
            du[3N+3*j+3] += mass[i+1] * dz / d^3 
        end
    end
end

We could have simplified the algorithm slightly by having both `i` and `j` loop over each planet.  Instead, the limits of each loop are set so that we only consider each pair once.  The most expensive part of calculating the acceleration is the `sqrt` function.  Thus, we speed up the calculation by about a factor of two. 

Next, we'll install all the packages needed for this lab (specified in the `test/Project.toml` file) and then load their modules.

In [ ]:
# If haven't already done this for ex1, then need to run the following once
using Pkg
Pkg.activate("test") # Tell Julia to look in test directory to find Project.toml
Pkg.instantiate()    # Tell Julia to install any packages needed for this assignment

In [ ]:
# Load packages that we'll be using for this exercise
using DifferentialEquations
using LinearAlgebra
using Plots
using Printf

Next, we will set the initial conditions for our integration.  For testing purposes, we want to create a system where we have a good understanding of the correct behavior.  We'll setup just two bodies, assign them masses analogous to the Sun and Jupiter, and place them on the $x$-axis, moving in the $x$-$y$ plane, so that they should follow a circular orbit

In [ ]:
# Set initial conditions
m = [1.0, 0.001]               # Masses: Sun and Jupiter
init_separation = 1.0          # separation: 1 AU
# Place star and planet on a nearly circular orbit
init_velocity = sqrt(sum(m)/init_separation) # Uniform circular motion
year = 2pi*sqrt(init_separation^3/sum(m))    # Kepler's third law
r_init = init_separation .* [-m[2]/sum(m), 0, 0, m[1]/sum(m), 0, 0 ]
v_init = init_velocity   .* [0, -m[2]/sum(m), 0, 0, m[1]/sum(m), 0 ]
# DifferentialEquations.jl wants the initial conditions as a single vector
u_init = vcat(r_init, v_init);  # concatenate positions & velocities

Rather than writing our own code to solve differential equations, we'll make use of the [DifferentialEquations.jl package](http://docs.juliadiffeq.org/stable/index.html).  It's big, powerful and complex.  Here we'll use just a few of it's features.  Below I illustrate how to create an "ODEProblem" object by providing the function to calculate derivatives (`gravity_as_first_order_system`), the initial conditions (`u_init`), a "Tuple" (here used to store an ordered-pair) with the start and stop time, and a list of fixed parameters (here the masses of the bodies, `m`).

In [ ]:
time_span = (0.0,4*year)       # Set how long to integrate for
# Setup the ODE problem, but don't actually solve it yet
prob = ODEProblem(gravity_as_first_order_system,u_init,time_span,m)

You can see from the output, that this returned a variable that is a [composite type](https://docs.julialang.org/en/v1/manual/types/index.html#Composite-Types-1), containing the information that we provided explicitly, as well as some additional variables that it set to default values for us.  For example, even though, we didn't explicitly specify that the `u` array would contain `Float64`'s, it figured that out and will make use of that information to compile code that is optimized for that type.

Before you execute the cells below to integrate the equations numerically, please think about what the trajectory of the planet _should_ look like if the equations were integrated perfectly.  

First, we'll integrate the differential equations using the simplest possible algorithm, [Euler's method](https://en.wikipedia.org/wiki/Euler_method).  
At each step of the integration, it sets $u(t_{i+1}) = u(t_i) + \Delta t \frac{du}{dt}(t_i,u_i)$.  

What do you expect the numerical solution will look like?

Ok, now you can go ahead and execute the cell below to integrate this system.  To keep things simple and make issues readily apparent, we'll manually set a fixed time step of one 36th of an orbit.  We'll use the `@time` macro to measure how long it take and how much memory is allocated.

In [ ]:
@time sol = solve(prob, Euler(), dt=year/36);

Wow, that took a lot of time and memory for such a short calculating.  I've been singing the praises of Julia as a very efficient language for scientific computing.  What happened here?
Let's try doing that that again (in the cell below, so you can compare).

In [ ]:
prob = ODEProblem(gravity_as_first_order_system,u_init,time_span,m)
@time sol = solve(prob, Euler(), dt=year/36);

a.  How does the run time and memory compare to the first time you executed the cell?  
What's changed?  
    
INSERT YOUR RESPONCE

Next, we'll plot the trajectory of the planet in the $x$-$y$ plane.  We could use the normal syntax to plot using Julia's [Plots.jl](http://docs.juliaplots.org/latest/) package.  However, the DifferentialEquations.jl package kindly provides a "recipie" that makes it even easier to plot the results of calling `DifferentialEquations.solve`.  We can call `plot` with just one arguments, the result of calling solve, and it would plot each of the 12 variables as a function of time.  I've added an optional argument to label the $x$-axis.

In [ ]:
plot(sol, xlabel="Time")

That's not the most intuitive way to visualize the results.  Below, I'll create a scatter plot and use `vars=(10,11)` to specify that we want the 10th and 11th variables (x and y position of body 2), so we can see the trajectory of the planet more clearly.  I've added some additional [optional arguements](http://docs.juliaplots.org/latest/attributes/) to make the plot a little prettier.

In [ ]:
scatter(sol,vars=(10,11), markersize = 0, xlabel="x", ylabel="y",
        title="Trajectory", legend = false)

b.  Is that what you expected?  What's wrong with this trajectory?  What could we do to numerically solve the system more accurately?  

INSERT YOUR RESPONCE

## Reducing the time-step
The Euler method has one parameter (`dt`) the ammount of simulation time that the system will be advanced in each step of the integrator.  Try adjusting the timestep in the cell below and observe how the trajectory changes.

In [ ]:
prob = ODEProblem(gravity_as_first_order_system,u_init,time_span,m)
sol = solve(prob, Euler(), dt=year/72);
scatter(sol,vars=(10,11),markersize = 0, xlabel="x", ylabel="y", 
        title="Trajectory", legend = false)

c.  Would reducing `dt` be a practical way to get accurate results with the Euler integrator?

## Increasing the order of the integrator

Euler's method is first-order.  We could try to improve on our result by using a second-order integration, variously known as modified Euler's method or Heun's method.  It first computes $\tilde{u}(t_{i+1}) = u(t_i) + \Delta t \frac{du}{dt}(t_i,u_i)$ and then improves the estimate with
$$u(t_{i+1}) = u(t_i) + \frac{\Delta t}{2} \left[ \frac{du}{dt}(t_i,u_i) + \frac{d\tilde{u}}{dt}(t_{i+1},u_i) \right].$$   
d.  What do expect the trajectory integrated with the improved Euler's method will look like?

INSERT YOUR RESPONCE

In [ ]:
prob = ODEProblem(gravity_as_first_order_system,u_init,time_span,m)
sol = solve(prob,Heun(),dt=year/36);
scatter(sol,vars=(10,11),markersize = 0, xlabel="x", ylabel="y", 
        title="Trajectory", legend = false)

e.  Did the results match your expectation?  If not, explain.  
How is the result better?  Based on your results, would you be comfortable using the algorithm above?   Why or why not?

INSERT YOUR RESPONCE

Now, let's try using the same algorithm, but for a longer integration.  Instead of a simulation of 4 orbits, we'll try integrating for a thousand orbits.  What do you expect the trajectory will look like?

In [ ]:
time_span = (0.0,1000*year) 
prob = ODEProblem(gravity_as_first_order_system,u_init,time_span,m)
sol = solve(prob,Heun(),dt=year/36,saveat=year);
scatter(sol,vars=(10,11),markersize = 0, xlabel="x", ylabel="y", 
        title="Trajectory", legend = false)

f.  Did the results match your expectation?  If not, explain.  
Based on these results, would you be comfortable using the algorithm above?   Why or why not?  How could we test whether an algorithm is providing acceptable results?

INSERT YOUR RESPONCE


## Quantifying  accuracey
In order to provide a more quantiative assessment of the integration accuracey, we can make use of our knowledge about the physics of this system.  Conservation laws dictate that the energy and angluar momentum should be conserved.  For a circular orbit, the star-planet separation should remain constant, and the orbital phase should increase linearly.  Below, I've written some helper functions that will allow you to more easily visualize how well each integration performs.

In [ ]:
"Calculate the phase of the orbit of the second body relative to x-axis (internal)"
function calc_angle(u::Vector)
    @assert length(u) >= 6  
    pl = 1  #  pl = 1 results in using the angle of first "planet"
    dx = u[3*pl+1]-u[1]
    dy = u[3*pl+2]-u[2]
    atan(dy,dx)
end

"""
   calc_phase_error(sol; phase_init = 0.0, year = 2pi)

Calculate a vector of the difference in orbital phase of the 
second body at each time relative to a linearly increasing phase.

Inputs:
   - sol is the result of DifferentialEquations.solve
   - year: time required for the phase to increase by 2pi
Outputs:
   - Vector of differences in phase
Assumptions:
   - sol.u contains an array of positions [x_1, y_1, z_1, x_2, y_2, z_2, ...]
   - sol.t contains an array of simulation times
"""
function calc_phase_error(sol::ODESolution; year = 2pi)
    pl = 1
    dtheta_dt = 2pi/year  
    phase_init = calc_angle(sol.u[1])  # Phase at first stored time
    delta = mod.(calc_angle.(sol.u).-(sol.t.-sol.t[1]).*dtheta_dt.-phase_init,2pi)
    # mod returns returns values in [0,2pi), but we want between [-pi,pi)
    delta[delta .< -pi] .+= 2pi
    delta[delta .>  pi] .-= 2pi
    delta
end

"Calculate the separation between first and second bodies"
function calc_separation(u::Vector)
    @assert length(u) >= 6
    pl = 1    # 1 results in separation between "star" and first "planet"
    dx = u[3*pl+1]-u[1]
    dy = u[3*pl+2]-u[2]    
    dz = u[3*pl+3]-u[3]
    sqrt(dx^2+dy^2+dz^2)
end

"""
   calc_energy(u::Vector, mass::Vector)

Calculate the energy of the system
Inputs:
- u: array of 3*N positions followed by 3*N velocities
  [x_1, y_1, z_1, x_2, y_2, z_2, ...., vx_1, vy_1, vz_1, ..., vz_N]
- mass: array of N masses
Assumes:
- Gravitational constants = 1
"""
function calc_energy(u::Vector, m::Vector)
    @assert length(u) == 6*length(m)
    @assert length(m) == 2  
    # Assumes 2 bodies each with 3 coordinates, so velocities begin at 7
    kinetic = 0.5*m[1]*sum(u[7:9].^2) + m[2]*sum(u[10:12].^2) 
    d = calc_separation(u)
    potential = -m[1]*m[2]/d
    kinetic + potential
end

"""
   calc_angular_momentum(u::Vector,m::Vector)

Calculate the angular momentum of the system"
Inputs:
- u: array of 3*N positions followed by 3*N velocities
  [x_1, y_1, z_1, x_2, y_2, z_2, ...., vx_1, vy_1, vz_1, ..., vz_N]
- mass: array of N masses
Assumes:
- Gravitational constants = 1
"""
function calc_angular_momentum(u::Vector,m::Vector)
    @assert length(u) == 6*length(m)
    N = length(m)
    @assert length(m) == 2  
    # Reshape u into a 2-d array that's 3x4
    # u[:,i] is the ith 3-vector. 
    # positions in i=[1,2] velocities in i[3,4]
    L  = m[1] * cross(reshape(u,3,4)[:,1], reshape(u,3,4)[:,3])
    L += m[2] * cross(reshape(u,3,4)[:,2], reshape(u,3,4)[:,4])
    L[3]
    #L

end

The first few times we did an integration and made plots, we did it one step at a time.  Going forward, you're going to try integrating this system many times changing the algorithm and a few parameters.  In order to make that more efficient (both for the computer and for you), we'll package all those stepts into a function, `make_test_plots_v1`.

In [ ]:
"""
   make_test_plots_v1(opts)

Integrate a two-body system and plot the change in energy, angular modmentum, 
radial separation of the planet and deviation of its phase from linear growth.

# Optional names arguements: (default value)
- alg: algorithm to use (DP8(); see http://docs.juliadiffeq.org/stable/solvers/ode_solve.html#Full-List-of-Methods-1)
- duration: duration to integrate in orbits (100)
- steps_per_orbit: number of time steps per orbit for Euler algorithm (ignored by other integrators) (1000)
- save_every_n_orbits: how often to store results for plotting (1)
- init_separation: initial separation (1)
- mass: masses of bodies ([1, 0.001])

"""
function make_test_plots_v1(; alg=DP8(), duration=100, steps_per_orbit=1000, 
        save_every_n_orbits=1, init_separation= 1, mass = [1.0, 0.001])
    @assert length(mass) == 2  # Phase and separation error only make sense for 1 planet
    # Setup initial conditions
    init_velocity = sqrt(sum(mass)/init_separation) # Uniform circular motion
    year = 2pi*sqrt(init_separation^3/sum(mass))    # Kepler's third law
    r_init = init_separation .* [-mass[2]/sum(mass), 0, 0, mass[1]/sum(mass), 0, 0 ]
    v_init = init_velocity   .* [0, -mass[2]/sum(mass), 0, 0, mass[1]/sum(mass), 0 ]
    # DifferentialEquations.jl wants the initial conditions as a single vector
    u_init = vcat(r_init, v_init);  # concatenate positions & velocities
    @assert length(u_init) == 6 * length(mass)
    year = 2pi*sqrt(init_separation^3/sum(mass))    # Kepler's third law
    time_span = (0.0,0.01*year) 
    prob = ODEProblem(gravity_as_first_order_system,u_init,time_span,mass)
    if alg==Euler() # Euler requires a specified time step dt
        # First do a very short integration to make sure code is compiled before timing
        sol = solve(prob,alg,dt=year/steps_per_orbit,saveat=year*save_every_n_orbits);
        time_span = (0.0,duration*year) 
        prob = ODEProblem(gravity_as_first_order_system,u_init,time_span,mass)
        # Now do the requested integration and time how long it takes
        @time sol = solve(prob,alg,dt=year/steps_per_orbit,saveat=year*save_every_n_orbits, force_dtmin=false);
    else # Other algorithms heuristically pick a timestep
        # First do a very short integration to make sure code is compiled before timing
        sol = solve(prob,alg,saveat=year*save_every_n_orbits);
        time_span = (0.0,duration*year) 
        prob = ODEProblem(gravity_as_first_order_system,u_init,time_span,mass)
        # Now do the requested integration and time how long it takes
        @time sol = solve(prob,alg,saveat=year*save_every_n_orbits);
    end
    separation_init = calc_separation(u_init)
    Lz_init = calc_angular_momentum(u_init,mass)
    E_init = calc_energy(u_init,mass)
    # Make plots
    plot_angle    = scatter(sol.t,calc_phase_error(sol,year=year), xlabel = "Time", ylabel = "Phase error", markersize=0, legend = false, grid=:no, xticks=0:round(duration*year/2,sigdigits=1):duration*year)
    plot_distance = scatter(sol.t,calc_separation.(sol.u).-separation_init, xlabel = "Time", ylabel = "Separation", markersize=0, legend = false, grid=:no, xticks=0:round(duration*year/2,sigdigits=1):duration*year)
    plot_energy   = scatter(sol.t,map(x->calc_energy(x,mass).-E_init,sol.u), xlabel = "Time", ylabel = "Energy Error", markersize=0, legend = false, grid=:no, xticks=0:round(duration*year/2,sigdigits=1):duration*year)
    plot_Lz       = scatter(sol.t,map(x->calc_angular_momentum(x,mass).-Lz_init,sol.u), xlabel = "Time", ylabel = "L_z Error", markersize=0, legend = false, grid=:no, xticks=0:round(duration*year/2,sigdigits=1):duration*year, yformatter=((x)->Printf.@sprintf "%0.1e" x))
    plot( plot_energy, plot_Lz, plot_distance, plot_angle, layout = (2,2) )
end

Now, we can easily integrate systems and inspect the results using just one line of code for each algorithm that we test.  For example:

In [ ]:
make_test_plots_v1(alg=Euler(), duration=100)

In [ ]:
make_test_plots_v1(alg=Heun(), steps_per_orbit=1000, duration=1000)

g.  Based on these plots, are you happy with the accuracy of the integration?  

Try experimenting with alternative integration algorithms by replacing `alg=Heun()` in the cell above with [other integration algorithms](http://docs.juliadiffeq.org/stable/solvers/ode_solve.html#Full-List-of-Methods-1) such as `Midpoint()`, `RK4()`, `Tsit5()`, `DP8()`.  (For higher-order algorithms, you may want to reduce the number of steps per orbits.)

In [ ]:
# SOLUTION
make_test_plots_v1(alg=Midpoint(), steps_per_orbit=1000, duration=1000)

In [ ]:
make_test_plots_v1(alg=RK4(), steps_per_orbit=1000, duration=1000)

In [ ]:
make_test_plots_v1(alg=Tsit5(), steps_per_orbit=1000, duration=1000)

In [ ]:
make_test_plots_v1(alg=DP8(), steps_per_orbit=36, duration=1000)

In [ ]:
make_test_plots_v1(alg=Vern8(), steps_per_orbit=1000, duration=1000)

h.  Did any of the integrators you tried perform acceptably?  If so, which?  

INSERT RESPONSE

## Choosing Appropriate Algorithms 
Next, we will rewrite the problem in a slightly different way that allows the DifferentialEquations.jl package to make use of the nice mathematical properties of a Hamiltonian system.  There are special mathematical properties of the N-body problem.
There are [specialized integration algorithms](http://docs.juliadiffeq.org/stable/solvers/dynamical_solve.html#Specialized-OrdinaryDiffEq.jl-Integrators-1) that can be applied when the derivative of the positions is proportional to the velocities and the derivative of the velocities does not depend on the velocities.  To make use of these algorithms, I've provided new functions that calculate the derivatives of the positions ("drift") separately from calculating the derivatives of the velocities ("kick").

In [ ]:
"""
   gravity_drift(du, v, u, mass, t)

Sets derivative of positions equal to velocities for a system of N massive particles.

# Inputs:  
- v: array of 3*N velocities
- u: array of 3*N positions 
  [x_1, y_1, z_1, x_2, y_2, z_2, ...., z_N]
- mass: array of N masses
- t: time

# Outputs:
- du: Aray of 3*N position derivatives

"""
function gravity_drift(du, v, u, mass, t)
    @assert length(du) == length(v) == length(u) == 3 * length(mass)
    N = length(mass)
    for k in 1:(3*N) 
       du[k] = v[k]     # derivative of positions is simply velocity
    end
end

"""
   gravity_kick(dv, v, u, mass, t)

Sets derivative of velocities equal to acceleration due to Newtonian gravity for a system of N massive particles.

# Inputs:  
- v: array of 3*N velocities
- u: array of 3*N positions 
  [x_1, y_1, z_1, x_2, y_2, z_2, ...., z_N]
- mass: array of N masses
- t: time

# Outputs:
- dv: Aray of 3*N velocity derivatives

# Assumptions:
- Gravitational constant = 1.  
Therefore, if masses are in solar masses and distances are in AU, 
then the time unit is such that one year = 2pi.

"""
function gravity_kick(dv, v, u, mass, t)
    @assert length(dv) == length(v) == length(u) == 3 * length(mass)
    N = length(mass)
    @assert 2 <= N      # require at least two massive bodies
    dv .= 0.0             # initialize accelerations to zero
    for i in 0:(N-2)      # loop over each pairs of bodies
        for j in (i+1):(N-1)
            dx = u[3*i+1] - u[3*j+1]  # displacements
            dy = u[3*i+2] - u[3*j+2]
            dz = u[3*i+3] - u[3*j+3]
            # calculate distance once per pair
            d = sqrt(dx^2+dy^2+dz^2)  
            # derivatives of velocities are accelerations 
            # acceleration on body i due to body j
            dv[3*i+1] -= mass[j+1] * dx / d^3 
            dv[3*i+2] -= mass[j+1] * dy / d^3 
            dv[3*i+3] -= mass[j+1] * dz / d^3 
            # acceleration on body j due to body i
            dv[3*j+1] += mass[i+1] * dx / d^3 
            dv[3*j+2] += mass[i+1] * dy / d^3 
            dv[3*j+3] += mass[i+1] * dz / d^3 
        end
    end
    dv
end

It turns out that DifferentialEquations stores the results in a slightly different format, so we need to make new versions of the function calc_phase_error, calc_energy, and calc_angular_momentum.

In [ ]:
"""
   calc_phase_error_v2(sol; phase_init = 0.0, year = 2pi)

Calculate a vector of the difference in orbital phase of the 
second body at each time relative to a linearly increasing phase.

Inputs:
   - sol is the result of DifferentialEquations.solve
   - year: time required for the phase to increase by 2pi
Outputs:
   - Vector of differences in phase
Assumptions:
   - Each sol.u[i].x[1] contains an array of positions at time i [x_1, y_1, z_1, x_2, y_2, z_2, ...]
   - sol.t contains an array of simulation times
"""
function calc_phase_error_v2(sol::ODESolution; year = 2pi)
    dtheta_dt = 2pi/year  
    phase_init = calc_angle(sol.u[1].x[1])
    delta = mod.(map(i->calc_angle(sol.u[i].x[1]),1:length(sol.u)).-(sol.t.-sol.t[1]).*dtheta_dt.-phase_init,2pi)
    # mod returns returns values in [0,2pi), but we want between [-pi,pi)
    delta[delta .< -pi] .+= 2pi
    delta[delta .>  pi] .-= 2pi
    delta
end

"Calculate the energy of the system"
function calc_energy(u::AbstractVector, v::AbstractVector, m::AbstractVector)
    @assert length(u) == length(v) == 3*length(m)
    @assert length(m) == 2 
    kinetic = 0.5*m[1]*sum(v[1:3].^2) + m[2]*sum(v[4:6].^2) 
    d = calc_separation(u)
    potential = -m[1]*m[2]/d
    kinetic + potential
end

"Calculate the angular momentum of the system"
function calc_angular_momentum(u::AbstractVector, v::AbstractVector, m::AbstractVector)
    @assert length(u) == length(v) == 3*length(m)
    N = length(m)
    @assert length(m) == 2 
    # cross works on 3-vectors, so use view(v,range) to provide view that looks like a 3-vector
    L  = m[1] * cross(view(u,1:3), view(v,1:3))
    L += m[2] * cross(view(u,4:6), view(v,4:6))
    L[3]
end

Now, we'll make a new function `make_test_plots_v2` that allows us to perform similar tests, but using the symplectic integration algorithms that are applicable to our problem.

In [ ]:
"""
   make_test_plots_v2(opts)

Integrate a two-body system and plot the change in energy, L_z, radial separation of the planet and deviation of phase from linear growth.
Setup for algorithms that uses knowledge that this is a Hamiltonian system

# Optional names arguements: (default value)
- alg: algorithm to use (KahanLi6(); see http://docs.juliadiffeq.org/stable/solvers/dynamical_solve.html#Symplectic-Integrators-1)
- duration: duration to integrate in orbits (100)
- steps_per_orbit: number of time steps per orbit (36)
- save_every_n_orbits: how often to store results for plotting (1)
- init_separation: initial separation (1)
- mass: masses of bodies ([1, 0.001])            
"""
function make_test_plots_v2(; duration=100, alg=KahanLi6(), steps_per_orbit=36, 
        save_every_n_orbits=1, init_separation = 1, mass=[1.0,0.001])
    @assert length(mass) == 2  # Phase and separation error only make sense for 1 planet
    # Setup initial conditions
    init_velocity = sqrt(sum(mass)/init_separation) # Uniform circular motion
    year = 2pi*sqrt(init_separation^3/sum(mass))    # Kepler's third law
    r_init = init_separation .* [-mass[2]/sum(mass), 0, 0, mass[1]/sum(mass), 0, 0 ]
    v_init = init_velocity   .* [0, -mass[2]/sum(mass), 0, 0, mass[1]/sum(mass), 0 ]
    @assert length(r_init) == length(v_init) == 3 * length(mass)
    # First do a very short integration to make sure code is compiled before timing
    time_span = (0.0,0.01*year) 
    prob = DynamicalODEProblem(gravity_kick,gravity_drift,v_init,r_init,time_span,m)
    sol = solve(prob,alg,dt=year/steps_per_orbit,saveat=year*save_every_n_orbits);
    # Now do the requested integration and time how long it takes
    time_span = (0.0,duration*year) 
    prob = DynamicalODEProblem(gravity_kick,gravity_drift,v_init,r_init,time_span,m)
    @time sol = solve(prob,alg,dt=year/steps_per_orbit,saveat=year*save_every_n_orbits);
    # Make plots
    angles = calc_phase_error_v2(sol,year=year)
    plot_angle     = scatter(sol.t,angles, xlabel = "Time", ylabel = "Phase error", markersize=0, legend = false, grid=:no, xticks=0:round(duration*year/2, sigdigits=1):duration*year, yformatter=((x)->Printf.@sprintf "%0.1e" x))
    separation_init = calc_separation(r_init)
    separations = map(i->calc_separation(sol.u[i].x[2]),1:length(sol.u)) .- separation_init

    plot_separations = scatter(sol.t,separations, xlabel = "Time", ylabel = "Separation", markersize=0, legend = false, grid=:no, xticks=0:round(duration*year/2, sigdigits=1):duration*year, yformatter=((x)->Printf.@sprintf "%0.1e" x))
    E_init = calc_energy(r_init,v_init,mass)
    energies = map(i->calc_energy(sol.u[i].x[2],sol.u[i].x[1],mass),1:length(sol.u)) .- E_init
    plot_energy   = scatter(sol.t,energies, xlabel = "Time", ylabel = "Energy Error", markersize=0, legend = false,
      grid=:no, xticks=0:round(duration*year/2, sigdigits=1):duration*year, yformatter=((x)->Printf.@sprintf "%0.1e" x ))
    Lz_init = calc_angular_momentum(r_init,v_init,mass)
    Lzs =  map(i->calc_angular_momentum(sol.u[i].x[2],sol.u[i].x[1],mass),1:length(sol.u)) .- Lz_init
    Lz_range = minimum(Lzs):maximum(Lzs)
    plot_Lz       = scatter(sol.t,Lzs, xlabel = "Time", ylabel = "L_z Error", markersize=0, legend = false, grid=:no, xticks=0:round(duration*year/2, sigdigits=1):duration*year, yformatter=((x)->Printf.@sprintf "%0.1e" x))
    println("# Final errors: E: ", energies[end], " Lz: ", Lzs[end], " r: ",separations[end], " θ: ", angles[end])
    plot( plot_energy, plot_Lz, plot_separations, plot_angle, layout = (2,2), fmt = :png )
end

First try a lower-order algorithm such as "[Leapfrog integration]https://en.wikipedia.org/wiki/Leapfrog_integration).  The `VerletLeapfrog()` integrator is second-order, but symplectic and time-reversible.  How do you expect the results using this integrator to differ from the results with previous integrators?

In [ ]:
make_test_plots_v2(alg=VerletLeapfrog(),steps_per_orbit=1000,duration=1000)

i.  How does the accuracy of the results with the Verlet Leapfrog integrator compare to previous results?  Based on these results, would you be comfortable using the algorithm above?   Why or why not?

INSERT RESPONCE

## Choosing efficient algorithms

j.  Try several of the [symplectic integrators](http://docs.juliadiffeq.org/stable/solvers/dynamical_solve.html#Symplectic-Integrators-1), such as `McAte3()`, `CalvoSanz4()`, `McAte5()`, `KahanLi6()`, `KahanLi8()`.  The number refers to the order of the integrator.  Compare the accuracy of the results with symplectic integrators of different orders.  
Also compare the time required.

In [ ]:
# INSERT RESPONCE

For an $n$th-order integrator the error term due to _truncation_ is generally of order $\Delta~t^{n+1}$.  That means that you could likely use a smaller number of steps_per_orbit with a higher-order algorithm to achieve a similar numerical precision.  

k.  Pick an accuracy target (e.g., $\Delta~$ or phase error at the end of the simulation).  Then tinker with the the number of steps per orbit, so that the results with different integrators achieve similar accuracy.  (No need to be super precise about this.  E.g., you might aim for the exponent to be the same, but not worry about the leading digit.)  Then compare the time required to achieve the target accuracy with different integrators.  Which algorithms and time-steps would you recommend for performing an N-body integration of the solar system?

INSERT RESPONCE

l.  How long would it take to integrate this system for $10^8$ orbits?  

INSERT RESPONCE

## Lessons learned

m.  Based on this lesson, what properties of an n-body integrator are most important for obtaining accurate scientific results when studying the long-term orbital of a planetary system?  

INSERT RESPONCE